Formula:
\begin{align}
\min \quad & \sum_{(i,j)\in A}d_{i,j}x_{i,j} \\
\text{s.t.} \quad & \sum_{(i,j)\in A : j=h} x_{i,j} = 1 && \forall \ h \in C \\
& \sum_{(i,j)\in A : i=h} x_{i,j} = 1 && \forall \ h \in C \\
& x_{i,j} = 1 \Rightarrow u_i + v_j = u_j && \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 \\
& x_{i,j} = 1 \Rightarrow w_i + p_j = w_j && \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 \\
& v_i \leq u_i \leq V && \forall \ i \in C \\
& p_i \leq w_i \leq P && \forall \ i \in C \\
& x_{i,j} \in \{0,1\} && \forall \ (i,j) \in A
\end{align}

In [1]:
import os
os.environ["SPARK_HOME"] = "D:\RISET\Spark\spark-3.1.1-bin-hadoop2.7"
from platform import python_version
print(python_version())

3.7.10


In [2]:
import numpy as np
import pandas as pd
import xlwings as xw
import folium
import docplex.mp.solution as mp_sol
from docplex.mp.model import Model
from geopy.distance import great_circle
import json

n = 50
customer = [i for i in range(1, n + 1)]
node = [0] + customer
arcos = [(i,j) for i in node for j in node if i != j]
f = open('pelanggan.json')
data = json.load(f)
# get customer demand and location data
df = pd.DataFrame(data)
# print(df)
# get distance data
distance = np.loadtxt('distance.txt')

vehicle_capacity = 250

In [3]:
mdl = Model('CVRP')

In [4]:
x = mdl.binary_var_dict(arcos, name='x')
u = mdl.continuous_var_dict(customer, name='u')
w = mdl.continuous_var_dict(customer, name='w')

$$\min \sum_{(i,j)\in A}d_{i,j}x_{i,j}$$

In [5]:
mdl.minimize(mdl.sum(distance[(i, j)] * x[(i, j)] for i, j in arcos))

$$ \sum_{(i,j)\in A : j=h} x_{i,j} = 1 \qquad \forall \ h \in C $$

In [6]:
for h in customer:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if i==h)==1, ctname='out_%d'%h)

$$ \sum_{(i,j)\in A : i=h} x_{i,j} = 1 \qquad \forall \ h \in C $$

In [7]:
for h in customer:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if j==h)==1, ctname='in_%d'%h)

$$ x_{i,j} = 1 \Rightarrow u_i + v_j = u_j \qquad \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 $$
$$ x_{i,j} = 1 \Rightarrow w_i + p_j = w_j \qquad \forall \ (i,j) \in A : i\neq 0 , \ j\neq 0 $$

In [8]:
for i, j in arcos:
    if i!=0 and j!=0:
        mdl.add_indicator(x[(i,j)], u[i] + df.demand[j] == u[j])

$$ v_i \leq u_i \leq V \qquad \forall \ i \in C $$
$$ p_i \leq w_i \leq P \qquad \forall \ i \in C $$

In [9]:
for i in customer:
    mdl.add_constraint(df.demand[i] <= u[i])
    mdl.add_constraint(u[i] <= vehicle_capacity)

In [10]:
mdl.parameters.timelimit = 60
# mdl.parameters.parallel = 1
# mdl.parameters.threads =  0
# mdl.parameters.benders.strategy = -1
# mdl.parameters.mip.limits.cutsfactor = 0
# mdl.parameters.mip.tolerances.mipgap = 0.1
# mdl.parameters.mip.strategy.branch = 0

###Menghitung Waktu
import time
start_timer = time.time()

solucion = mdl.solve(log_output=True)

###Menghitung Waktu
done_timer = time.time()
elapsed = done_timer - start_timer

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Found incumbent of value 461315.000000 after 0.00 sec. (0.20 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 100 rows and 50 columns.
MIP Presolve modified 1225 coefficients.
Aggregator did 1225 substitutions.
Reduced MIP has 1325 rows, 3825 columns, and 8675 nonzeros.
Reduced MIP has 2550 binaries, 0 generals, 0 SOSs, and 2450 indicators.
Presolve time = 0.09 sec. (18.18 ticks)
Probing time = 0.23 sec. (40.72 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1325 rows, 3825 columns, and 8675 nonzeros.
Reduced MIP has 2550 binaries, 0 generals, 0 SOSs, and 2450 indicators.
Presolve time = 0.05 sec. (6.47 ticks)
Probing time = 0.01 sec. (3.25 ticks)
Clique table members: 1325.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up t

In [11]:
mdl.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [12]:
# Encontrar las rutas por separado
route = []
total_dis = []
for h in customer:
    if x[(0, h)].solution_value > 0.9:
        arcos_route = [(0, h)]
        item_dis = distance[0][h]
        j = h
        while j!=0:
            i = j
            for k in node:
                if i!=k and x[(i, k)].solution_value > 0.9:
                    j = k
                    arcos_route.append((i, j))
                    item_dis+= distance[i][j]
                    break
        route.append(arcos_route)
        total_dis.append(item_dis)

In [13]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))
print(elapsed)

Rute 0
[(0, 1), (1, 2), (2, 24), (24, 9), (9, 3), (3, 8), (8, 0)] -> 8531.0
Rute 1
[(0, 10), (10, 23), (23, 6), (6, 22), (22, 0)] -> 5893.0
Rute 2
[(0, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 4), (4, 40), (40, 0)] -> 10808.0
Rute 3
[(0, 21), (21, 11), (11, 50), (50, 25), (25, 26), (26, 0)] -> 9820.0
Rute 4
[(0, 31), (31, 32), (32, 42), (42, 41), (41, 30), (30, 0)] -> 34120.0
Rute 5
[(0, 34), (34, 48), (48, 43), (43, 44), (44, 36), (36, 47), (47, 0)] -> 20173.0
Rute 6
[(0, 35), (35, 46), (46, 45), (45, 12), (12, 7), (7, 13), (13, 0)] -> 13818.0
Rute 7
[(0, 38), (38, 37), (37, 39), (39, 5), (5, 15), (15, 14), (14, 0)] -> 11278.0
Rute 8
[(0, 49), (49, 33), (33, 27), (27, 29), (29, 28), (28, 0)] -> 28404.0
Total Distace : 142845.0
60.14341998100281
